In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
import nibabel as nib
from os import listdir
from os.path import isfile, join
from scipy import signal
import lib

In [2]:
# current path
cur_path = os.getcwd()
# file name and path
file = "BB_003_3T_cmrr_mbep2d_diff_0pt2_1_task_18s_off_12s_on_20200710180104_8"
file_name = cur_path + "/" + str(file) + "/dfMRI_raw.nii.gz"

In [3]:
# topup exception
# analysis_id = 26 < 36
# => phase-encoding dimension
PE_dim = 'AP'

# current path
cur_path = os.getcwd()
# file name and path
file = "BB_003_3T_cmrr_mbep2d_diff_0pt2_1_task_18s_off_12s_on_20200710180104_8"
file_name = cur_path + "/" + str(file) + "/dfMRI_raw.nii.gz"
# loading nifti object/header
raw_data_lin = nib.load(file_name)
header_lin  = raw_data_lin.header
raw_data_lin = raw_data_lin.get_fdata()

file = "BB_003_3T_cmrr_mbep2d_diff_0pt2_1_task_18s_off_12s_on_20200710180104_8"
file_name = cur_path + "/" + str(file) + "/BB_003_3T_cmrr_mbep2d_diff_0pt2_1_revPE_20200710180104_9.nii.gz"
# loading nifti object/header
raw_data_revPE = nib.load(file_name)
header_revPE  = raw_data_revPE.header
raw_data_revPE = raw_data_revPE.get_fdata()

No_slice_lin = header_lin['dim'][3]
No_slice_revPE = header_revPE['dim'][3]

In [4]:
No_slice_lin,No_slice_revPE

(14, 16)

In [5]:
img_linPE_b0 = nib.Nifti1Image(raw_data_lin[...,np.arange(0, raw_data_lin.shape[-1], 2)], 
                    np.eye(4), header_lin)
nib.save(img_linPE_b0, cur_path + f'/data/topup/dfmri_linPE_b0_vols.nii.gz')  
img_revPE = nib.Nifti1Image(raw_data_revPE, np.eye(4), header_revPE)
nib.save(img_revPE, cur_path + f'/data/topup/dfmri_revPE_vols.nii.gz')  

In [6]:
dfmri_linPE_b0_vols = nib.load( cur_path + f'/data/topup/dfmri_linPE_b0_vols.nii.gz')
dfmri_linPE_b0_vols = dfmri_linPE_b0_vols.get_fdata()
assert np.array_equal(dfmri_linPE_b0_vols, raw_data_lin[...,np.arange(0, raw_data_lin.shape[-1], 2)]), "The linPE_b0_vols is wrong"

# Apply -Tmean on linPE
linPE_b0_one_vol = np.mean(dfmri_linPE_b0_vols, axis=3)
linPE_b0_one_vol_bck = linPE_b0_one_vol.copy()
img_linPE_b0_one_vol = nib.Nifti1Image(linPE_b0_one_vol, np.eye(4))
nib.save(img_linPE_b0_one_vol, cur_path + f'/data/topup/dfmri_linPE_b0_one_vol.nii.gz') 

# Apply -Tmean on revPE
revPE_one_vol = np.mean(raw_data_revPE, axis=3)
revPE_one_vol_bck = revPE_one_vol.copy()
img_revPE_one_vol = nib.Nifti1Image(revPE_one_vol, np.eye(4))
nib.save(img_revPE_one_vol, cur_path + f'/data/topup/dfmri_revPE_one_vol.nii.gz')

## Check that -Tmean is the mean according to axis 3 (time/b values)

In [7]:
import os

for PE in ['linPE_b0', 'revPE']:
    os.system('fslmaths '
            f'{cur_path}/data/topup/dfmri_{PE}_vols.nii.gz' 
            f' -Tmean {cur_path}/data/topup/fsl_tman_test.nii.gz')

    test_fsl_tmean = nib.load(f'{cur_path}/data/topup/fsl_tman_test.nii.gz')
    test_fsl_tmean = test_fsl_tmean.get_fdata()

    test_numpy = nib.load(cur_path + f'/data/topup/dfmri_{PE}_one_vol.nii.gz')
    test_numpy = test_numpy.get_fdata()
    assert np.array_equal(test_fsl_tmean, test_numpy), f"There is a difference between fslmaths -Tmean and np.mean({PE}, axis=3)"

    os.remove(f'{cur_path}/data/topup/fsl_tman_test.nii.gz') 

    del test_fsl_tmean, test_numpy

## Check that fslroi is equivalent as image cropping

In [38]:
linPE_b0_one_vol = nib.load(cur_path + f'/data/topup/dfmri_linPE_b0_one_vol.nii.gz')
linPE_b0_one_vol = linPE_b0_one_vol.get_fdata()

revPE_one_vol = nib.load(cur_path + f'/data/topup/dfmri_revPE_one_vol.nii.gz')
revPE_one_vol = revPE_one_vol.get_fdata()

# If the number of z slice differs between linPE and revPE, 
# crop the larger image
if No_slice_lin < No_slice_revPE :
    start_idx = int((No_slice_revPE-No_slice_lin)/2)
    stop_idx = (int((No_slice_revPE-No_slice_lin)/2)+No_slice_lin)
    z_size = No_slice_lin
    revPE_one_vol = revPE_one_vol[:,:, start_idx:stop_idx] 
elif No_slice_lin > No_slice_revPE :
    start_idx = int((No_slice_lin-No_slice_revPE)/2)
    stop_idx = (int((No_slice_lin-No_slice_revPE)/2)+No_slice_revPE)
    z_size = No_slice_revPE
    linPE_b0_one_vol = linPE_b0_one_vol[:,:, start_idx:stop_idx] 
else :
    print("linPE and revPE have the same number of z slices")

img_linPE_b0_one_vol = nib.Nifti1Image(linPE_b0_one_vol, np.eye(4))
nib.save(img_linPE_b0_one_vol, cur_path + f'/data/topup/dfmri_linPE_b0_one_vol_cropped.nii.gz') 

img_revPE_one_vol = nib.Nifti1Image(revPE_one_vol, np.eye(4))
nib.save(img_revPE_one_vol, cur_path + f'/data/topup/dfmri_revPE_one_vol_cropped.nii.gz')


In [40]:
if No_slice_lin < No_slice_revPE :
    to_crop = 'revPE'
    
elif No_slice_lin > No_slice_revPE :
    to_crop = 'linPE_b0'

os.system('fslroi '
          f'{cur_path}/data/topup/dfmri_{to_crop}_one_vol.nii.gz ' 
          f'{cur_path}/data/topup/dfmri_{to_crop}_fslroi.nii.gz ' 
          '0 -1 0 -1 '
          f'{int(abs(No_slice_revPE-No_slice_lin)/2)} '
          f'{z_size} 0 -1')

test_fslroi = nib.load(f'{cur_path}/data/topup/dfmri_{to_crop}_fslroi.nii.gz')
test_fslroi = test_fslroi.get_fdata()
test_numpy = nib.load(f'{cur_path}/data/topup/dfmri_{to_crop}_one_vol_cropped.nii.gz')
test_numpy = test_numpy.get_fdata()

assert np.array_equal(test_fslroi, test_numpy), f"The cropped version of {to_crop} is different from the fslroi one"

del test_fslroi, test_numpy


In [42]:
dfmri_linPE_b0_vols.shape

(116, 116, 14, 300)

## Check that fslmerge -t is actually merging 2 volumes according to the time (axis 3)

In [49]:

linPE_revPE = np.concatenate((linPE_b0_one_vol[...,np.newaxis], revPE_one_vol[...,np.newaxis]), axis=3)

os.system('fslmerge -t '
         f'{cur_path}/data/topup/test_fslmerge.nii.gz '
         f'{cur_path}/data/topup/dfmri_linPE_b0_one_vol.nii.gz '
         f'{cur_path}/data/topup/dfmri_{to_crop}_one_vol_cropped.nii.gz ')

test_fslmerge = nib.load(f'{cur_path}/data/topup/test_fslmerge.nii.gz')
test_fslmerge = test_fslmerge.get_fdata()
assert np.array_equal(test_fslmerge, linPE_revPE), "Fslmerge and np.concatenate lead to different linPE_revPE array"

del test_fslmerge

## Calculate deformation field topup with linPE_revPE volume

### 1. Pad the image on z-axis

In [57]:
def image_padding(input_img, axis_padding=2):
    """
    Function that padds the image according to an axis
    Input:
        input_img : 4D numpy array
        axis_padding : axis along which to do the padding. Either 0, 1 or 2
    Output:
        The padded volume : 4D numpy array
    """
    match axis_padding:
        case 0:
            first_plane = input_img[0,...]
            first_plane = first_plane[np.newaxis,...]
            last_plane = input_img[-1,...]
            last_plane = last_plane[np.newaxis,...]
        case 1:
            first_plane = input_img[:,0,:,:]
            first_plane = first_plane[:,np.newaxis,:,:]
            last_plane = input_img[:,-1,:,:]
            last_plane = last_plane[:,np.newaxis,:,:]
        case 2:
            first_plane = input_img[:,:,0,:]
            first_plane = first_plane[:,:,np.newaxis,:]
            last_plane = input_img[:,:,-1,:]
            last_plane = last_plane[:,:,np.newaxis,:]
        case _:
            print("The padding can only be performed on axis = 0, 1 or 2")
        
    return np.concatenate((first_plane, input_img, last_plane), axis=axis_padding)

## Check that fslroi is equivalent as image cropping

In [58]:
linPE_revPE_zpadded = image_padding(linPE_revPE, axis_padding=2)
img_linPE_revPE = nib.Nifti1Image(linPE_revPE_zpadded, np.eye(4))
nib.save(img_linPE_revPE, cur_path + f'/data/topup/dfmri_linPE_revPE_padded.nii.gz')  

(116, 116, 16, 2)


### 2. Calculate the deformation field

In [59]:
os.system('topup --imain='
          f'{cur_path}/data/topup/dfmri_linPE_revPE_padded.nii.gz '
          f'--datain={cur_path}/acqp_{PE_dim}.txt '
          f'--out={cur_path}/data/topup/topup_res '
          f'--iout={cur_path}/data/topup/dfmri_linPE_revPE_tu.nii.gz '
          f'--config={cur_path}/topup.cnf -v '
          f'--fout={cur_path}/data/topup/fieldval.nii.gz')    

SSD = 508.61	n = 19488	Reg = 0	Cost = 508.61
SSD = 315.244	n = 16082	Reg = 29.0677	Cost = 344.312
SSD = 205.057	n = 13962	Reg = 38.6681	Cost = 243.725
SSD = 162.138	n = 12298	Reg = 41.2061	Cost = 203.344
SSD = 144.939	n = 11890	Reg = 46.6018	Cost = 191.541
SSD = 140.255	n = 11436	Reg = 49.4857	Cost = 189.74
***Going to next resolution level***
SSD = 212.372	n = 11436	Reg = 16.2206	Cost = 228.593
SSD = 158.88	n = 13183	Reg = 28.2187	Cost = 187.099
SSD = 114.389	n = 14575	Reg = 36.9542	Cost = 151.343
SSD = 91.7825	n = 14869	Reg = 43.9435	Cost = 135.726
SSD = 80.7009	n = 15013	Reg = 48.7508	Cost = 129.452
SSD = 74.0995	n = 15222	Reg = 50.829	Cost = 124.928
***Going to next resolution level***
SSD = 180.086	n = 15222	Reg = 15.1061	Cost = 195.192
SSD = 91.9409	n = 15492	Reg = 18.8345	Cost = 110.775
SSD = 57.9623	n = 16011	Reg = 21.4292	Cost = 79.3915
SSD = 50.8433	n = 16872	Reg = 23.1303	Cost = 73.9736
SSD = 42.1422	n = 18035	Reg = 19.8303	Cost = 61.9725
SSD = 37.56	n = 19442	Reg = 17.876	C

0

### 3. Pad the data on which you want to apply topup on the z-axis as well

In [63]:
file_name = cur_path + "/data/test_mrdegibbs.nii.gz"
# loading nifti object
dfmri_degibbs = nib.load(file_name)
dfmri_degibbs = dfmri_degibbs.get_fdata()

dfmri_degibbs_zpadded = image_padding(dfmri_degibbs, axis_padding=2)
img_degibbs = nib.Nifti1Image(dfmri_degibbs_zpadded, np.eye(4))
nib.save(img_degibbs,  f'{cur_path}/data/topup/dfmri_degibbs_padded.nii.gz')  

### 4. Apply topup

In [65]:
os.system('applytopup -i '
          f'{cur_path}/data/topup/dfmri_degibbs_padded.nii.gz '
          '--inindex=1 '
          f'--datain={cur_path}/acqp_{PE_dim}.txt '
          f'--topup={cur_path}/data/topup/topup_res '
          f'--out={cur_path}/data/topup/dfmri_deggibs_padded_uw '
          '--method=jac')

0

### 5. Remove z-padding

In [67]:
dfmri_deggibs_padded_uw = nib.load(f'{cur_path}/data/topup/dfmri_deggibs_padded_uw.nii.gz')
dfmri_deggibs_padded_uw = dfmri_deggibs_padded_uw.get_fdata()
dfmri_deggibs_sdc = dfmri_deggibs_padded_uw[:,:,1:z_size+1,:]


dfmri_deggibs_sdc = nib.Nifti1Image(dfmri_deggibs_sdc, np.eye(4))
nib.save(dfmri_deggibs_sdc, f'{cur_path}/data/topup/dfmri_degibbs_sdc.nii.gz')  